## GDC RNA-Seq Tool Example
### This notebook lets users upload a manifest file, and download Merged RNA-Seq Data

In [ ]:
import base64
from IPython.display import HTML, display
from ipyupload import FileUpload
import subprocess
import pandas as pd

def Write_Manifest_File(w):
    File_Name = list(w.value.keys())[0]
    file = open(File_Name,'w') 
    file.write(w.value[File_Name]['content'].decode('UTF-8')) 
    file.close()
    return File_Name

## Step 1: Upload a Manifest File

In [ ]:
w = FileUpload()
display('Upload a Manifest File:' , w)

## Step 2: Run GDC-RNASeq-Tool On Manifest

In [ ]:
Manifest_Name = Write_Manifest_File(w)
print('Running GDC-RNA-Seq Tool on Manifest File called ' + Manifest_Name + '....')
p = subprocess.run(['python','gdc-rnaseq-tool.py',Manifest_Name,'--hugo'],stdout=subprocess.PIPE)
print('Finished Running')

## Step 3: Download Merged Data

In [ ]:
# Read in Data Frame that was created
File_Location = p.stdout.decode('UTF-8').split('\n')[1].split(': ')[1]

Files = []
for line in p.stdout.decode('UTF-8').split('\n'):
    if 'Creating merged' in line:
        Files.append(line.split(' ')[7])

def create_download_link( df, title , filename ): 
    csv = df.to_csv(sep='\t')
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/tsv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

for file in Files:
    df = pd.read_csv(File_Location + file,sep='\t')
    display(create_download_link(df, file, file))